In [1]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from config import *
from pprint import pprint

/home/azureuser/Ambarish/sentence-llama-index/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-01 16:23:36.587 | WARNING  | fastembed.embedding:<module>:7 - DefaultEmbedding, FlagEmbedding, JinaEmbedding are deprecated.Use from fastembed import TextEmbedding instead.


In [2]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [3]:
client = QdrantClient(URL,port=6333)

In [4]:
vector_store = QdrantVectorStore(client=client, collection_name=COLLECTION_NAME)

In [5]:
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.node_parser import SentenceSplitter

# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

In [6]:
# base node parser is a sentence splitter
text_splitter = SentenceSplitter()

In [7]:
from llama_index.llms.azure_openai import AzureOpenAI
llm = AzureOpenAI(
    model="gpt-35-turbo",
    deployment_name=deployment_id_gpt4,
    api_key=key,
    azure_endpoint=endpoint,
    api_version=api_version,
)

In [8]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [9]:
embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-mpnet-base-v2",max_length=512
)

In [10]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.text_splitter = text_splitter

In [11]:
from llama_index.core  import VectorStoreIndex

In [12]:
sentence_index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

In [13]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor

query_engine = sentence_index.as_query_engine(
    similarity_top_k=2,
    # the target key defaults to `window` to match the node_parser's default
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)

In [14]:
#query = "Describe about Tata Steel Kalinganagar?"
query = "What should the government do about road safety?"

In [15]:
response = query_engine.query(query)

In [16]:
pprint(response.response)

('The government should make road safety a political priority, develop a '
 'multidisciplinary approach, appoint a lead agency for road safety, set '
 'appropriate road safety targets, establish national road safety plans to '
 'achieve them, support the creation of safety advocacy groups, create budgets '
 'for road safety, and increase investment in demonstrably effective road '
 'safety activities.')


In [17]:
query = "What is  ENERGY INTENSITY AT TSJ?"
query = "What is the Golden Quadrilateral ?"

In [18]:
query

'What is the Golden Quadrilateral ?'

In [19]:
response = query_engine.query(query)

In [20]:
pprint(response.response)

('The Golden Quadrilateral is a component of the National Highways Development '
 'Project (NHDP) in India, which spans a total length of 5,846 km and connects '
 'Delhi, Mumbai, Chennai, Kolkata, and Delhi.')
